# Teleportation Kata

**Teleportation** quantum kata is a series of exercises designed to get you familiar with programming in Q#. It covers the quantum teleportation protocol which allows you to communicate a quantum state using only classical communication and previously shared quantum entanglement.

 - Teleportation is described in [this Wikipedia article](https://en.wikipedia.org/wiki/Quantum_teleportation).
 - An interactive demonstration can be found [on the Wolfram Demonstrations Project](http://demonstrations.wolfram.com/QuantumTeleportation/).

Each task is wrapped in one operation preceded by the description of the task.  Your goal is to fill in the blank (marked with `// ...` comment) with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

In [ ]:
open Microsoft.Quantum.Diagnostics;

In [ ]:
%config dump.basisStateLabelingConvention = "Bitstring"
%config dump.truncationThreshold = 0.01
%config dump.truncateSmallAmplitudes = true
%config  

Configuration key,Value
dump.basisStateLabelingConvention,"""Bitstring"""
dump.truncationThreshold,0.01
dump.truncateSmallAmplitudes,true


## Part I. Standard Teleportation
We split the teleportation protocol into several steps:
    
* Preparation (creating the entangled pair of qubits that are sent to Alice and Bob).
* Sending the message (Alice's task): Entangling the message qubit with Alice's qubit and extracting two classical bits to be sent to Bob.
* Reconstructing the message (Bob's task): Using the two classical bits Bob received from Alice to get Bob's qubit into the state in which the message qubit had been originally.  Finally, we compose these steps into the complete teleportation protocol.

### Task 1.1. Entangled pair

**Input:** two qubits qAlice and qBob, each in $|0\rangle$ state.

**Goal:**  prepare a Bell state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ on these qubits.

> In the context of the quantum teleportation protocol, this is the preparation step: qubits qAlice and qBob will be sent to Alice and Bob, respectively.

In [ ]:
%kata T11_Entangle 

operation Entangle (qAlice : Qubit, qBob : Qubit) : Unit {
    // ...
    H(qAlice);
    CNOT(qAlice, qBob);
}

Success!

### Task 1.2. Send the message (Alice's task)

Entangle the message qubit with Alice's qubit and extract two classical bits to be sent to Bob.

**Inputs**: 
1. Alice's part of the entangled pair of qubits qAlice.
2. The message qubit qMessage.

**Output**:
Two classical bits Alice will send to Bob via classical channel as a tuple of Bool values. The first bit in the tuple should hold the result of measurement of the message qubit, the second bit - the result of measurement of Alice's qubit.
Represent measurement result 'One' as `true` and 'Zero' as `false`.  The state of the qubits in the end of the operation doesn't matter.

In [ ]:
%kata T12_SendMessage 

operation SendMessage (qAlice : Qubit, qMessage : Qubit) : (Bool, Bool) {
    // ...
    CNOT(qMessage, qAlice);
    H(qMessage);
    let mqa = M(qAlice) == One;
    let mqm = M(qMessage) == One;
    return (mqm, mqa);
}

Success!

### Task 1.3. Reconstruct the message (Bob's task)

Transform Bob's qubit into the required state using the two classical bits received from Alice.

**Inputs**: 
1. Bob's part of the entangled pair of qubits qBob.
2. The tuple of classical bits received from Alice, in the format used in task 1.2.

**Goal** : Transform Bob's qubit qBob into the state in which the message qubit had been originally.

In [ ]:
%kata T13_ReconstructMessage 

operation ReconstructMessage (qBob : Qubit, (b1 : Bool, b2 : Bool)) : Unit {
    // ...
    if b1 == false and b2 == false {
        
    } elif b1 == false and b2 == true {
        X(qBob);
    } elif b1 == true  and b2 == false {
        Z(qBob);
    } elif b1 == true  and b2 == true {
        X(qBob);
        Z(qBob);
    }
}

Success!

### Task 1.4. Standard teleportation protocol

Put together the steps implemented in tasks 1.1 - 1.3 to implement the full teleportation protocol.

**Inputs:**

1. The two qubits qAlice and qBob in $|0\rangle$ state.
2. The message qubit qMessage in the state $|\psi\rangle$ to be teleported.

**Goal:** Transform Bob's qubit qBob into the state $|\psi\rangle$. The state of the qubits qAlice and qMessage in the end of the operation doesn't matter.

In [ ]:
%kata T14_StandardTeleport

operation StandardTeleport (qAlice : Qubit, qBob : Qubit, qMessage : Qubit) : Unit {
    // ...
    Entangle(qAlice, qBob);
    let bits = SendMessage(qAlice, qMessage);
    ReconstructMessage(qBob, bits);
}

Success!

### Task 1.5. Prepare a state and send it as a message (Alice's task)

Given a Pauli basis along with a state `true` as 'One' or `false` as 'Zero', prepare a message qubit, entangle it with Alice's qubit, and extract two classical bits to be sent to Bob.

**Inputs:** 

1. Alice's part of the entangled pair of qubits qAlice.
2. A PauliX, PauliY, or PauliZ basis in which the message qubit should be prepared
3. A Bool indicating the eigenstate in which the message qubit should be prepared

**Output:** 

Two classical bits Alice will send to Bob via classical channel as a tuple of Bool values.  The first bit in the tuple should hold the result of measurement of the message qubit, the second bit - the result of measurement of Alice's qubit.  Represent measurement result 'One' as `true` and 'Zero' as `false`. The state of the qubit qAlice in the end of the operation doesn't matter.

In [ ]:
operation PreparePauli (q: Qubit, basis: Pauli, state: Bool): Unit is Adj+Ctl {
    if state { X(q); }
    if basis == PauliX {
        H(q);
    } elif basis == PauliY {
        H(q);
        S(q);
    } elif basis == PauliZ {
        
    } else {
        fail "??";
    }
}

PreparePauli

In [ ]:
%kata T15_PrepareAndSendMessage

operation PrepareAndSendMessage (qAlice : Qubit, basis : Pauli, state : Bool) : (Bool, Bool) {
    // ...
    use qMsg = Qubit();
    PreparePauli(qMsg, basis, state);
    CNOT(qMsg, qAlice);
    H(qMsg);
    // return SendMessage(qAlice, qMsg);
    //DumpMachine();
    let mqa = M(qAlice) == One;
    let mqm = M(qMsg) == One;
    
    return (mqm, mqa);
}

Success!

### Task 1.6. Reconstruct and measure the message state (Bob's task)

Transform Bob's qubit into the required state using the two classical bits received from Alice and measure it in the same basis in which she prepared the message.
    
**Inputs:**

1. Bob's part of the entangled pair of qubits qBob.
2. The tuple of classical bits received from Alice, in the format used in task 1.5.
3. The PauliX, PauliY, or PauliZ basis in which the message qubit was originally prepared.

**Output:** A Bool indicating the eigenstate in which the message qubit was prepared, 'One' as `true` and 'Zero' as `false`.  The state of the qubit qBob in the end of the operation doesn't matter.

> To get the output, transform Bob's qubit qBob into the state in which the message qubit was originally prepared, then measure it. 

In [ ]:
%kata T16_ReconstructAndMeasureMessage

operation ReconstructAndMeasureMessage (qBob : Qubit, (b1 : Bool, b2 : Bool), basis : Pauli) : Bool {
        
    // ...
    
    if b2 { X(qBob); }
    if b1 { Z(qBob); }
    Adjoint PreparePauli(qBob, basis, false);
    
    return M(qBob) == One;
}

Success!

### Task 1.7. Testing standard quantum teleportation

**Goal:** Test that the `StandardTeleport` operation from task 1.4 is able to successfully teleport the states $|0\rangle$ and $|1\rangle$, as well as superposition states such as $\frac{1}{\sqrt{2}} \big(|0\rangle + |1\rangle\big)$, $\frac{1}{\sqrt{2}} \big(|0\rangle - |1\rangle\big)$, $\frac{1}{\sqrt{2}} \big(|0\rangle + i|1\rangle\big)$ and $\frac{1}{\sqrt{2}} \big(|0\rangle - i|1\rangle\big)$.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_StandardTeleport` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_StandardTeleport`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.

<details>
  <summary><b>Need a hint? Click here</b</summary>
  You may find your answers for 1.5 and 1.6 useful
</details>

In [ ]:
open Microsoft.Quantum.Diagnostics;

operation Run_StandardTeleport () : String {
    // ...
    use qMessage = Qubit();
    use qAlice = Qubit();
    use qBob = Qubit();
    
    Rx(1.0, qMessage);
    
    Message("Before Teleportation");
    DumpMachine();
    StandardTeleport(qAlice, qBob, qMessage);
    Message("After Teleporation");
    DumpMachine();
    
    return "Success!";
}

Run_StandardTeleport

In [ ]:
%simulate Run_StandardTeleport

Before Teleportation


|000⟩	0.8775825618903728 + 0𝑖
|100⟩	0 + -0.479425538604203𝑖

After Teleporation


|010⟩	0.8775825618903726 + 0𝑖
|011⟩	0 + -0.47942553860420295𝑖

Source,Callable
(notebook),Run_StandardTeleport


Released qubits are not in zero state.


## Part II. Teleportation using different entangled pair

In this section we will take a look at the changes in the reconstruction process (Bob's task) if the qubits shared between Alice and Bob are entangled in a different state.  Alice's part of the protocol remains the same in all tasks.  
As a reminder, the standard teleportation protocol requires shared qubits in state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} \big(|00\rangle + |11\rangle\big)$.

In each task, the inputs are
1. Bob's part of the entangled pair of qubits qBob.
2. The tuple of classical bits received from Alice, in the format used in task 1.2.

The goal is to transform Bob's qubit qBob into the state in which the message qubit had been originally.

### Task 2.1. Reconstruct the message if the entangled qubits were in the state $|\Phi^{-}\rangle = \frac{1}{\sqrt{2}} \big(|00\rangle - |11\rangle\big)$

In [ ]:
%kata T21_ReconstructMessage_PhiMinus

operation ReconstructMessage_PhiMinus (qBob : Qubit, (b1 : Bool, b2 : Bool)) : Unit {
    // ...
    if not b1 and not b2 {
        Z(qBob);
    } elif not b1 and b2 {
        Z(qBob);
        X(qBob);
    } elif b1 and not b2 {
        
    } elif b1 and b2 {
        Z(qBob);
        X(qBob);
        Z(qBob);
    }
}

Success!

### Task 2.2. Reconstruct the message if the entangled qubits were in the state $|\Psi^{+}\rangle = \frac{1}{\sqrt{2}} \big(|01\rangle + |10\rangle\big)$

In [ ]:
%kata T22_ReconstructMessage_PsiPlus

operation ReconstructMessage_PsiPlus (qBob : Qubit, (b1 : Bool, b2 : Bool)) : Unit {
    // ...
    if not b1 and not b2 {
        X(qBob);
    } elif not b1 and b2 {
        
    } elif b1 and not b2 {
        X(qBob);
        Z(qBob);
    } elif b1 and b2 {
        Z(qBob);
    }
}

Success!

### Task 2.3. Reconstruct the message if the entangled qubits were in the state $|\Psi^{-}\rangle = \frac{1}{\sqrt{2}} \big(|01\rangle - |10\rangle\big)$

In [ ]:
%kata T23_ReconstructMessage_PsiMinus

operation ReconstructMessage_PsiMinus (qBob : Qubit, (b1 : Bool, b2 : Bool)) : Unit {
    // ...
    if not b1 and not b2 {
        X(qBob);
        Z(qBob);
    } elif not b1 and b2 {
        X(qBob);
        Z(qBob);
        X(qBob);
    } elif b1 and not b2 {
        X(qBob);
    } elif b1 and b2 {
        Z(qBob);
        X(qBob);
        Z(qBob);
        X(qBob);
    }
}


Success!

## Part III. Principle of deferred measurement

The principle of deferred measurement claims that measurements can be moved from an intermediate stage of a quantum circuit to the end of the circuit.  If the measurement results are used to perform classically controlled operations, they can be replaced by controlled quantum operations.

### Task 3.1. Measurement-free teleportation.

In this task we will apply this principle to the teleportation circuit.

**Inputs:** 

1. The two qubits qAlice and qBob in $|\Phi^{+}\rangle$ state.
2. The message qubit qMessage in the state $|\psi\rangle$ to be teleported.

**Goal:** transform Bob's qubit qBob into the state $|\psi\rangle$ using no measurements.  At the end of the operation qubits qAlice and qMessage should not be entangled with qBob.

In [ ]:
%kata T31_MeasurementFreeTeleport

operation MeasurementFreeTeleport (qAlice : Qubit, qBob : Qubit, qMessage : Qubit) : Unit {
    // ... 
    // DumpMachine();
    CNOT(qMessage, qAlice);
    H(qMessage);
    CNOT(qAlice, qBob);
    Controlled Z([qMessage], qBob);
    // DumpMachine();
}

Success!

## Part IV. Teleportation with three entangled qubits

Quantum teleportation using entangled states other than Bell pairs is also feasible.  Here we look at just one of many possible schemes - in it a state is transferred from Alice to a third participant Charlie, but this may only be accomplished if Charlie
has the trust of the second participant Bob.

### Task 4.1. Entangled trio

**Inputs:** three qubits qAlice, qBob, and qCharlie, each in $|0\rangle$ state.

**Goal:** create an entangled state $|\Psi^{3}\rangle = \frac{1}{2} \big(|000\rangle + |011\rangle + |101\rangle + |110\rangle\big)$ on these qubits.

In the context of the quantum teleportation protocol, this is the preparation step: qubits qAlice, qBob, and qCharlie will be sent to Alice, Bob, and Charlie respectively.

In [ ]:
%kata T41_EntangleThreeQubits

operation EntangleThreeQubits (qAlice : Qubit, qBob : Qubit, qCharlie : Qubit) : Unit {
    // ...
    H(qAlice);
    H(qBob);
    CNOT(qAlice, qCharlie);
    CNOT(qBob, qCharlie);
}

Success!

### Task 4.2. Reconstruct the message (Charlie's task)

Alice has a message qubit in the state $|\psi\rangle$  to be teleported, she has entangled it with her own qubit from $|\Psi^{3}\rangle$ in the same manner as task 1.2 and extracted two classical bits in order to send them to Charlie. Bob has also measured his own qubit from $|\Psi^{3}\rangle$ and sent Charlie the result. Transform Charlie's qubit into the required state using the two classical bits received from Alice, and the one classical bit received from Bob.

**Inputs:** 
1. Charlie's part of the entangled trio of qubits qCharlie.
2. The tuple of classical bits received from Alice, in the format used in task 1.2.
3. A classical bit resulting from the measurement of Bob's qubit.

**Goal:** Transform Charlie's qubit qCharlie into the state in which the message qubit had been originally.

In [ ]:
%kata T42_ReconstructMessageWhenThreeEntangledQubits

operation ReconstructMessageWhenThreeEntangledQubits (qCharlie : Qubit, (b1 : Bool, b2 : Bool), b3 : Bool) : Unit {
    // ...
    if b3 {
        X(qCharlie);
    }
    if b2 {
        X(qCharlie);
    }
    if b1 {
        Z(qCharlie);
    }
}

Success!